In [2]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px



# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

#drop down list
dlist= spacex_df['Launch Site'].unique().tolist()
dlist+=['ALL sites']
# Create a dash application
app = dash.Dash(__name__)

#pie df data
successfail_dummies = pd.get_dummies(spacex_df['class'])
combinedf = pd.concat([successfail_dummies,spacex_df['Launch Site']],axis =1)
combinedf.rename(columns = {0:'Fail', 1:'Success'}, inplace = True)
groupdf= combinedf.groupby('Launch Site')


# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)

                                dcc.Dropdown(
                                    id = "input",
                                    options=[
                                        {'label': i, 'value': i} for i in dlist
                                    ],
                                    value='ALL sites'
                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',min=min_payload,max=max_payload,step=1,value=[min_payload,max_payload]), html.Br(),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
#@app.callback(
    #Output("success-pie-chart", "figure"), 
   # [Input("input", "success-pie-chart"), 
   #  Input("success-pie-chart", "success-pie-chart")])
@app.callback( Output(component_id='success-pie-chart', component_property='figure'),
               Input(component_id='input', component_property='value'))

def generate_chart(values):
    if values == 'ALL sites':
        successcounts = groupdf['Success'].sum().sum()
        failcounts = groupdf['Fail'].sum().sum()
    else:
        successcounts = groupdf.get_group(values)['Success'].sum()
        failcounts = groupdf.get_group(values)['Fail'].sum()
    fig = px.pie(values=[successcounts,failcounts],names=['Success', 'Fail'],title='Success vs. Failed') #values=values)
    return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback( Output(component_id='success-payload-scatter-chart', component_property='figure'),
               Input(component_id='payload-slider', component_property='value'),
               Input(component_id='input', component_property='value'))

def scatter_chart(values,drop):

    if drop == 'ALL sites':
        rangedf= spacex_df[spacex_df['Payload Mass (kg)'].between(values[0],values[1])]
    else:
        selectdf = spacex_df[spacex_df['Launch Site']==drop]
        rangedf= selectdf[selectdf['Payload Mass (kg)'].between(values[0],values[1])]
    fig = px.scatter(
        data_frame=rangedf,
        x='Payload Mass (kg)',
        y="class",
    )
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server()


--2021-08-13 16:06:57--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3033 (3.0K) [text/csv]
Saving to: 'spacex_launch_dash.csv'

     0K ..                                                    100% 3.53G=0s

2021-08-13 16:06:57 (3.53 GB/s) - 'spacex_launch_dash.csv' saved [3033/3033]

